In [81]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.optimize import curve_fit
import json

In [82]:
df = pd.read_csv("SwimDataTop50.csv")

In [83]:
# Function to parse time strings with variable formats
def parse_custom_time(time_str):
    format = len(str(time_str).split(":"))

    if format == 1:
        return pd.to_timedelta(f'00:00:{time_str}')
    elif format == 2:
        return pd.to_timedelta(f'00:{time_str}')
    elif format == 3:
        return pd.to_timedelta(time_str)

# Convert 'time' column to timedelta 
df['time'] = df['time'].apply(parse_custom_time)
df['time_seconds'] = df['time'].dt.total_seconds()
df['speed'] = df['distance'] / df['time_seconds']

# Group the data by 'surname', 'firstname', 'track length', 'technique'
grouped_data = df.groupby(['surname', 'firstname', 'track length', 'technique'])
grouped_data.size()

surname    firstname  track length  technique    
ARMSTRONG  Hunter     25            Freistil         3
                                    Rücken           2
                      50            Freistil         4
                                    Rücken           3
                                    Schmetterling    1
                                                    ..
ZHAO       Jing       25            Lagen            4
                                    Rücken           6
                      50            Freistil         2
                                    Lagen            4
                                    Rücken           6
Length: 388, dtype: int64

In [84]:
def fit_rational_function(x, a, b, c):
    return 1 / (a * x + b) + c

In [85]:
# Function to filter data for a specific swimmer and plot the rational function fit
def plot_rational_function_for_swimmer(df, firstname, lastname, track_length, technique):
    values = {}
    returndict = {}
    map = {
        'F': 'Freistil',
        'R': 'Rücken',
        'B': 'Brust',
        'S': 'Schmetterling',
        'L': 'Lagen'
    }
    technique = map[technique]
    
    filtered_df = df[(df['firstname'] == firstname) & (df['surname'] == lastname) & (df['track length'] == track_length) & (df['technique'] == technique)]

    if filtered_df.empty:
        print(f"No data found for {firstname} {lastname}.")
        return

    params, covariance = curve_fit(fit_rational_function, filtered_df['distance'], filtered_df['speed'], method="dogbox")

    x_range = np.linspace(0, 2000, 200)
    y_pred = fit_rational_function(x_range, *params)

    for X, Y in zip(x_range, y_pred):
        returndict[X] = Y

    for distance, speed in zip(filtered_df['distance'], filtered_df['speed']):
        values[distance] = speed

    print(returndict)

    data = {
        "pred_values": returndict,
        "mes_values": values
    }

    return data

    


In [86]:
plot_rational_function_for_swimmer(df, firstname='Kathleen', lastname='LEDECKY', track_length=25, technique='F')

{0.0: 2.030435753274389, 10.050251256281408: 2.001268232277227, 20.100502512562816: 1.9759467217462166, 30.150753768844226: 1.953757573588228, 40.20100502512563: 1.9341534168645984, 50.25125628140704: 1.9167073959800875, 60.30150753768845: 1.9010817335537182, 70.35175879396985: 1.887005646016809, 80.40201005025126: 1.874259519057839, 90.45226130653268: 1.8626633662104517, 100.50251256281408: 1.8520682764538592, 110.55276381909549: 1.8423499850789216, 120.6030150753769: 1.8334039772663824, 130.6532663316583: 1.8251417143681747, 140.7035175879397: 1.8174876936170956, 150.75376884422113: 1.8103771341480632, 160.80402010050253: 1.8037541390213798, 170.85427135678393: 1.7975702227981927, 180.90452261306535: 1.7917831225681828, 190.95477386934675: 1.7863558307475098, 201.00502512562815: 1.7812558028426557, 211.05527638190958: 1.7764543043345575, 221.10552763819098: 1.7719258689916304, 231.15577889447238: 1.7676478470454027, 241.2060301507538: 1.763600026304426, 251.2562814070352: 1.759764312

{'pred_values': {0.0: 2.030435753274389,
  10.050251256281408: 2.001268232277227,
  20.100502512562816: 1.9759467217462166,
  30.150753768844226: 1.953757573588228,
  40.20100502512563: 1.9341534168645984,
  50.25125628140704: 1.9167073959800875,
  60.30150753768845: 1.9010817335537182,
  70.35175879396985: 1.887005646016809,
  80.40201005025126: 1.874259519057839,
  90.45226130653268: 1.8626633662104517,
  100.50251256281408: 1.8520682764538592,
  110.55276381909549: 1.8423499850789216,
  120.6030150753769: 1.8334039772663824,
  130.6532663316583: 1.8251417143681747,
  140.7035175879397: 1.8174876936170956,
  150.75376884422113: 1.8103771341480632,
  160.80402010050253: 1.8037541390213798,
  170.85427135678393: 1.7975702227981927,
  180.90452261306535: 1.7917831225681828,
  190.95477386934675: 1.7863558307475098,
  201.00502512562815: 1.7812558028426557,
  211.05527638190958: 1.7764543043345575,
  221.10552763819098: 1.7719258689916304,
  231.15577889447238: 1.7676478470454027,
  241.